(c) 2016 - present. Enplus Advisors, Inc.

In [1]:
import numpy as np
import pandas as pd

# Programming with Data<br>Foundations of Python and Pandas 
# Lesson 2: Data Frames

## What is a DataFrame?

The workhorse of `pandas`

Similar to a table in SQL or a list of named records

Named after `data.frame` in the `R` language, from which it was inspired.

## Creating a DataFrame

Several ways to [create](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) a DataFrame:

* From a dict of lists

### From a dict of lists

In [2]:
# From a dict of lists
df = pd.DataFrame({
    'ticker': ['AAPL', 'AAPL', 'MSFT', 'IBM', 'YHOO'],
    'date': ['2015-12-30', '2015-12-31', '2015-12-30', '2015-12-30', '2015-12-30'],
    'open': [426.23, 427.81, 42.3, 101.65, 35.53]
})
df

,ticker,date,open
0,AAPL,2015-12-30,426.23
1,AAPL,2015-12-31,427.81
2,MSFT,2015-12-30,42.30
3,IBM,2015-12-30,101.65
4,YHOO,2015-12-30,35.53


### From a list of dicts

Several ways to [create](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) a DataFrame:

* From a dict of lists

* From a `list` of `dict`s

In [3]:
pd.DataFrame([
    {'ticker': 'AAPL', 'date': '2015-12-30', 'open': 426.23},
    {'ticker': 'AAPL', 'date': '2015-12-31', 'open': 427.81},
    {'ticker': 'MSFT', 'open': 42.3, 'date': '2015-12-30'}
])


,date,open,ticker
0,2015-12-30,426.23,AAPL
1,2015-12-31,427.81,AAPL
2,2015-12-30,42.30,MSFT


### From another DataFrame

Several ways to [create](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) a DataFrame:

* From a dict of lists

* From a `list` of `dict`s

* From another DataFrame

In [4]:
pd.DataFrame(df, columns=['ticker', 'date', 'open', 'close'])

,ticker,date,open,close
0,AAPL,2015-12-30,426.23,NaN
1,AAPL,2015-12-31,427.81,NaN
2,MSFT,2015-12-30,42.30,NaN
3,IBM,2015-12-30,101.65,NaN
4,YHOO,2015-12-30,35.53,NaN


Notice that the `close` column is all `NaN`

## Single Axis Selection - Columns

Uses `dict` style lookup, e.g. `df['col1']` or attribute lookup, e.g. `df.col1`

Filter either rows or columns, but not both. 
(unless you pass a DataFrame to select individual cells).

Attribute and scalar indexing of columns returns a `Series`, NOT a `DataFrame`

### Attribute lookup

Attribute access based on column names. Only works for
valid attribute names (no spaces, keywords)

In [5]:
df.ticker

0    AAPL
1    AAPL
2    MSFT
3     IBM
4    YHOO
Name: ticker, dtype: object

### `dict` lookup

Dictionary-style (`__getitem__`) lookup is more flexible, supports
names that aren't valid attributes

In [6]:
df['ticker name'] = df['ticker']
df['ticker name']

0    AAPL
1    AAPL
2    MSFT
3     IBM
4    YHOO
Name: ticker name, dtype: object

In [7]:
del df['ticker name']

### Multiple columns

If you want to retrieve a single or multi-column DataFrame or multiple you
must pass a `list`.

In [8]:
df[['ticker']]

,ticker
0,AAPL
1,AAPL
2,MSFT
3,IBM
4,YHOO


## Single Axis Selection - Rows

Works similarly to `Series` logical indexing.

In [9]:
idx = [True, True, False, True, True]
df[idx]

,ticker,date,open
0,AAPL,2015-12-30,426.23
1,AAPL,2015-12-31,427.81
3,IBM,2015-12-30,101.65
4,YHOO,2015-12-30,35.53


## Single Axis Selection - Summary

When the index is:

* list[bool] operates on rows
* str or list[str] operates on columns
* `slice` then operates on the columns
* `DataFrame` operates cell-by-cell

## Multi-Axis Selection

The most common situation is logical indexing on the rows and
label indexing on the columns using `loc`.

In [10]:
idx = [True, True, False, True, True]
df.loc[idx, ['date', 'open']]

,date,open
0,2015-12-30,426.23
1,2015-12-31,427.81
3,2015-12-30,101.65
4,2015-12-30,35.53


### Row Labels

May select by _label_ on both rows and columns.

We haven't set an index on `df` so it has the default integer index.
Let's set one now.

In [11]:
df

,ticker,date,open
0,AAPL,2015-12-30,426.23
1,AAPL,2015-12-31,427.81
2,MSFT,2015-12-30,42.30
3,IBM,2015-12-30,101.65
4,YHOO,2015-12-30,35.53


In [12]:
# Note that `df1` is a copy of `df`
df1 = df.set_index('ticker')

### Row Labels (cont)

In [13]:
df1

,date,open
ticker,,
AAPL,2015-12-30,426.23
AAPL,2015-12-31,427.81
MSFT,2015-12-30,42.30
IBM,2015-12-30,101.65
YHOO,2015-12-30,35.53


The tickers are no longer part of the values of the `DataFrame`

Consequently, we can use them for index lookups.

### Multi-Axis Selection

In [14]:
# Select by row label
df1.loc['MSFT']

date    2015-12-30
open          42.3
Name: MSFT, dtype: object

Defaults to all columns, but I prefer explicit selection.
Easier to figure out what your code is doing.

In [15]:
# explicitly require all columns and return a dataframe
df1.loc[['MSFT'], :]

,date,open
ticker,,
MSFT,2015-12-30,42.3


### Multi-Axis Gotcha

Rows may be returned as either `Series` or `DataFrame` by using `loc`.

In [16]:
df1.loc['AAPL', :]

,date,open
ticker,,
AAPL,2015-12-30,426.23
AAPL,2015-12-31,427.81


Have to be careful of whether a `Series` or `DataFrame` is returned.

## Modifying a DataFrame

**Goals**: 

* Create a new column `close` with the same values as `open`
* Set `close` to `430.35` when `date == '2015-12-31'`

### Use `loc` or `iloc` for inplace modification

In [17]:
df1a = df1.copy() # don't modify df1
df1a['close'] = df1a['open']
df1a.loc[df1a.date == '2015-12-31', 'close'] = 5000.
df1a

,date,open,close
ticker,,,
AAPL,2015-12-30,426.23,426.23
AAPL,2015-12-31,427.81,5000.00
MSFT,2015-12-30,42.30,42.30
IBM,2015-12-30,101.65,101.65
YHOO,2015-12-30,35.53,35.53


### How not to modify a DataFrame

* `pandas` returns a "view" of a `DataFrame` when selecting a subset
* View does not create a copy though **cannot** be used to modify the original

In [18]:
df1b = df1.copy()
df1b['close'] = df1b['open']
df1b_view = df1b[df1b.date == '2015-12-31']
df1b_view

,date,open,close
ticker,,,
AAPL,2015-12-31,427.81,427.81


### How not to modify a DataFrame (cont)

Creates a copy when you assign to a view and emits a warning

In [19]:
df1b_view['close'] = 5000

/Users/daniel/anaconda3/envs/pydata/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df1b_view

,date,open,close
ticker,,,
AAPL,2015-12-31,427.81,5000


### The original `DataFrame` is unchanged.

In [21]:
df1b

,date,open,close
ticker,,,
AAPL,2015-12-30,426.23,426.23
AAPL,2015-12-31,427.81,427.81
MSFT,2015-12-30,42.30,42.30
IBM,2015-12-30,101.65,101.65
YHOO,2015-12-30,35.53,35.53


### Subsets without warnings

If you want to work with the subset, make a copy:

In [22]:
# What you want is
df2 = df1[df1.date != '2015-12-31'].copy()
df2

,date,open
ticker,,
AAPL,2015-12-30,426.23
MSFT,2015-12-30,42.30
IBM,2015-12-30,101.65
YHOO,2015-12-30,35.53


In [23]:
df2['close'] = 1
df2

,date,open,close
ticker,,,
AAPL,2015-12-30,426.23,1
MSFT,2015-12-30,42.30,1
IBM,2015-12-30,101.65,1
YHOO,2015-12-30,35.53,1


## Assignment Gotchas

TODO: Maybe show new Int64 type here?

### Types of NaNs

In [24]:
# May create NAs with `np.nan`, None, or float('nan')
df2['close'] = np.nan

In [25]:
df2['close'] = None
df2.close.dtype

dtype('O')

### Silently drop data when adding columns

Assigning a `Series` to a `DataFrame` column does an implicit left-join.

In [26]:
closes = pd.Series({'AAPL': 430.0, 'MSFT': 43.5, 'SP5': 1263.5})
df2['close'] = closes
df2

,date,open,close
ticker,,,
AAPL,2015-12-30,426.23,430.0
MSFT,2015-12-30,42.30,43.5
IBM,2015-12-30,101.65,NaN
YHOO,2015-12-30,35.53,NaN


### Avoiding automatic alignment

Use a sequence (`list`, `tuple`) or a `numpy` array if you don't
want automatic alignment.

In [27]:
x = pd.Series([1, 2, 3, 4], index=list('abcd'))
df2['close'] = x
df2

,date,open,close
ticker,,,
AAPL,2015-12-30,426.23,NaN
MSFT,2015-12-30,42.30,NaN
IBM,2015-12-30,101.65,NaN
YHOO,2015-12-30,35.53,NaN


In [28]:
df2['close'] = x.array
df2

,date,open,close
ticker,,,
AAPL,2015-12-30,426.23,1
MSFT,2015-12-30,42.30,2
IBM,2015-12-30,101.65,3
YHOO,2015-12-30,35.53,4


## Sorting DataFrames

In [29]:
df2.sort_index()

,date,open,close
ticker,,,
AAPL,2015-12-30,426.23,1
IBM,2015-12-30,101.65,3
MSFT,2015-12-30,42.30,2
YHOO,2015-12-30,35.53,4


In [30]:
df2.sort_values('open')

,date,open,close
ticker,,,
YHOO,2015-12-30,35.53,4
MSFT,2015-12-30,42.30,2
IBM,2015-12-30,101.65,3
AAPL,2015-12-30,426.23,1
